2-WARMUP

In [ ]:
!pip uninstall -y -q unsloth
!pip install --upgrade --quiet --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --quiet trl datasets accelerate bitsandbytes faiss-cpu pandas tqdm

from unsloth import FastLanguageModel, is_bfloat16_supported
import torch, random, os, urllib.request, json
from tqdm import tqdm

MODEL_NAME   = "unsloth/Llama-3.2-1B-Instruct"
MAX_SEQ_LEN  = 2048

base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name     = MODEL_NAME,
    max_seq_length = MAX_SEQ_LEN,
    load_in_4bit   = True,
    dtype          = None,   # auto
)
base_model.eval()

lora_model = FastLanguageModel.get_peft_model(
    base_model,
    r                         = 16,
    target_modules            = ["q_proj","k_proj","v_proj","o_proj",
                                 "gate_proj","up_proj","down_proj"],
    lora_alpha                = 16,
    lora_dropout              = 0,
    bias                      = "none",
    use_gradient_checkpointing= "unsloth",
    random_state              = 3407,
    use_rslora                = False,
)

from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

tokenizer = get_chat_template(base_tokenizer, chat_template="llama-3.1")

def format_entries(entry):
    conversations = [
        [
            {"role": "user",      "content": q},
            {"role": "assistant", "content": sql}
        ]
        for q, sql in zip(entry["question"], entry["query"])
    ]
    texts = [
        tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt=False)
        for conv in conversations
    ]
    return {
        "text": texts,
        "db_id": entry["db_id"],
        "question": entry["question"],
        "query": entry["query"]
    }

train_ds = load_dataset("xlangai/spider", split="train")
val_ds   = load_dataset("xlangai/spider", split="validation")
train_ds = train_ds.map(format_entries, batched=True)
val_ds   = val_ds.map(format_entries,   batched=True)

TABLES_URL  = "https://raw.githubusercontent.com/taoyds/spider/master/tables.json"
TABLES_FILE = "./tables.json"
if not os.path.exists(TABLES_FILE):
    urllib.request.urlretrieve(TABLES_URL, TABLES_FILE)

with open(TABLES_FILE, "r") as f:
    tables_list = json.load(f)
tables = {t["db_id"]: t for t in tables_list}

def build_schema(db):
    t = tables[db]
    col2tbl = {
        i: (t["column_names_original"][i][0], t["column_names_original"][i][1])
        for i in range(len(t["column_types"]))
    }
    parts = []
    for tidx, tname in enumerate(t["table_names_original"]):
        cols = [
            f'"{c}" {t["column_types"][cid]}'
            for cid,(tbl,c) in col2tbl.items()
            if tbl==tidx and c!="*"
        ]
        pk = [
            f'"{t["column_names_original"][cid][1]}"'
            for cid in t["primary_keys"]
            if col2tbl[cid][0]==tidx
        ]
        parts.append(
            f'"{tname}" {", ".join(cols)}' +
            (f', primary key: {" ".join(pk)}' if pk else "")
        )
    return " [SEP] ".join(parts)

import torch
from transformers import DataCollatorForSeq2Seq, TrainingArguments
from trl import SFTTrainer

EVAL_SUBSET = 100  # تعداد نمونهٔ ارزیابی

def jaccard_similarity(a: str, b: str) -> float:
    a_set, b_set = set(a.lower().split()), set(b.lower().split())
    if not a_set and not b_set:
        return 1.0
    return len(a_set & b_set) / len(a_set | b_set)

@torch.no_grad()
def evaluate_metrics(model, tokenizer, dataset, num_samples=EVAL_SUBSET):
    model.eval()
    subset  = dataset.shuffle(seed=42).select(range(num_samples))
    correct = 0
    jac_sum = 0.0

    for item in tqdm(subset, total=num_samples):
        schema_txt = build_schema(item["db_id"])
        prompt_text = f"Schema: {schema_txt}\nQuestion: {item['question']}"

        prompt_ids = tokenizer.apply_chat_template(
            [{"role":"user","content": prompt_text}],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        attn_mask = torch.ones_like(prompt_ids)

        out = model.generate(
            input_ids      = prompt_ids,
            attention_mask = attn_mask,
            max_new_tokens = 64,
            do_sample      = False,
        )

        generated = out[0, prompt_ids.shape[1]:]
        pred_sql  = tokenizer.decode(generated, skip_special_tokens=True)
        gt_sql    = item["query"]

        if pred_sql.strip() == gt_sql.strip():
            correct += 1
        jac_sum += jaccard_similarity(pred_sql, gt_sql)

    acc = correct / num_samples
    jac = jac_sum / num_samples
    return acc, jac

WARMUP_LIST = [0, 200, 1000]
MAX_STEPS   = 60

for w_steps in WARMUP_LIST:
    print(f"\n========== Training with warmup_steps = {w_steps} ==========")
    trainer = SFTTrainer(
        model              = lora_model,
        tokenizer          = tokenizer,
        train_dataset      = train_ds,
        dataset_text_field = "text",
        max_seq_length     = MAX_SEQ_LEN,
        data_collator      = DataCollatorForSeq2Seq(tokenizer=tokenizer),
        dataset_num_proc   = 2,
        packing            = False,
        args = TrainingArguments(
            per_device_train_batch_size = 2,
            gradient_accumulation_steps = 4,
            weight_decay                = 0.01,
            lr_scheduler_type           = "linear",
            seed                        = 3407,
            output_dir                  = f"outputs_w{w_steps}",
            report_to                   = "none",
            warmup_steps                = w_steps,
            max_steps                   = MAX_STEPS,
            learning_rate               = 2e-4,
            fp16                        = not is_bfloat16_supported(),
            bf16                        = is_bfloat16_supported(),
            logging_steps               = 1,
            optim                       = "adamw_8bit",
        ),
    )

    trainer.train()

    acc, jac = evaluate_metrics(lora_model, tokenizer, val_ds)
    print(f">>> Accuracy  (exact-match)    : {acc:.2%}")
    print(f">>> Jaccard Similarity (avg.) : {jac:.2%}")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 240.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 380.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 347.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 376.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 370.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 274.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 165.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 218.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 180.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3

model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

100%|██████████| 100/100 [02:15<00:00,  1.35s/it]


>>> Accuracy  (exact-match)    : 7.00%
>>> Jaccard Similarity (avg.) : 53.78%

========== Training with warmup_steps = 1000 ==========


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


100%|██████████| 100/100 [02:22<00:00,  1.42s/it]

>>> Accuracy  (exact-match)    : 4.00%
>>> Jaccard Similarity (avg.) : 51.91%


temp

In [ ]:
!pip uninstall -y -q unsloth
!pip install --upgrade --quiet --no-cache-dir \
    "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --quiet trl datasets accelerate bitsandbytes pandas matplotlib

from unsloth import FastLanguageModel, is_bfloat16_supported
import torch, os, json, urllib.request
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

MODEL_NAME   = "unsloth/Llama-3.2-1B-Instruct"
MAX_SEQ_LEN  = 2048

base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name     = MODEL_NAME,
    max_seq_length = MAX_SEQ_LEN,
    load_in_4bit   = True,
)
base_model.eval()

lora_model = FastLanguageModel.get_peft_model(
    base_model,
    r                         = 16,
    target_modules            = [
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj"
    ],
    lora_alpha                = 16,
    lora_dropout              = 0,
    bias                      = "none",
    use_gradient_checkpointing= "unsloth",
    random_state              = 3407,
    use_rslora                = False,
)

from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

tokenizer = get_chat_template(base_tokenizer, chat_template="llama-3.1")

def format_entries(entry):
    convs = [
        [
            {"role":"user","content": q},
            {"role":"assistant","content": sql}
        ]
        for q, sql in zip(entry["question"], entry["query"])
    ]
    texts = [
        tokenizer.apply_chat_template(c, tokenize=False, add_generation_prompt=False)
        for c in convs
    ]
    return {
        "text": texts,
        "db_id": entry["db_id"],
        "question": entry["question"],
        "query": entry["query"]
    }

train_ds = load_dataset("xlangai/spider", split="train")
val_ds   = load_dataset("xlangai/spider", split="validation")
train_ds = train_ds.map(format_entries, batched=True)
val_ds   = val_ds.map(format_entries,   batched=True)

TABLES_URL  = "https://raw.githubusercontent.com/taoyds/spider/master/tables.json"
TABLES_FILE = "./tables.json"
if not os.path.exists(TABLES_FILE):
    urllib.request.urlretrieve(TABLES_URL, TABLES_FILE)

with open(TABLES_FILE) as f:
    tables_list = json.load(f)
tables = {t["db_id"]: t for t in tables_list}

def build_schema(db_id):
    t = tables[db_id]
    col2tbl = {
        i: (t["column_names_original"][i][0], t["column_names_original"][i][1])
        for i in range(len(t["column_types"]))
    }
    parts = []
    for tidx, tname in enumerate(t["table_names_original"]):
        cols = [
            f'"{c}" {t["column_types"][cid]}'
            for cid,(tbl,c) in col2tbl.items()
            if tbl == tidx and c != "*"
        ]
        pk = [
            f'"{t["column_names_original"][cid][1]}"'
            for cid in t["primary_keys"]
            if col2tbl[cid][0] == tidx
        ]
        parts.append(
            f'"{tname}" ' + ", ".join(cols) +
            (f', primary key: {" ".join(pk)}' if pk else "")
        )
    return " [SEP] ".join(parts)

EVAL_SUBSET  = 100                
TEMPERATURES = [None, 0.2, 0.4, 0.6, 1.0]   

def jaccard_similarity(a: str, b: str) -> float:
    a_set, b_set = set(a.lower().split()), set(b.lower().split())
    if not a_set and not b_set:
        return 1.0
    return len(a_set & b_set) / len(a_set | b_set)

@torch.no_grad()
def evaluate_metrics(model, tokenizer, dataset,
                     temp=None, num_samples=EVAL_SUBSET):
    model.eval()
    subset  = dataset.shuffle(seed=42).select(range(num_samples))
    correct = 0
    jac_sum = 0.0

    for item in tqdm(subset, total=num_samples):
        # ساخت پرامپت: درج اسکیما قبل از پرسش
        schema_txt = build_schema(item["db_id"])
        system_content = f"Database schema:\n{schema_txt}"
        user_content   = item["question"]

        prompt_ids = tokenizer.apply_chat_template(
            [
                {"role":"system", "content": system_content},
                {"role":"user",   "content": user_content}
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        out = model.generate(
            input_ids      = prompt_ids,
            attention_mask = torch.ones_like(prompt_ids),
            max_new_tokens = 64,
            do_sample      = temp is not None,
            temperature    = temp if temp is not None else 1.0,
        )

        gen_tokens = out[0, prompt_ids.shape[1]:]
        pred_sql   = tokenizer.decode(gen_tokens, skip_special_tokens=True)
        gt_sql     = item["query"]

        if pred_sql.strip() == gt_sql.strip():
            correct += 1
        jac_sum += jaccard_similarity(pred_sql, gt_sql)

    return correct / num_samples, jac_sum / num_samples

from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer

MAX_STEPS     = 120
WARMUP_FIXED  = 200

trainer = SFTTrainer(
    model              = lora_model,
    tokenizer          = tokenizer,
    train_dataset      = train_ds,
    dataset_text_field = "text",
    max_seq_length     = MAX_SEQ_LEN,
    data_collator      = DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc   = 2,
    packing            = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        weight_decay                = 0.01,
        lr_scheduler_type           = "linear",
        seed                        = 3407,
        output_dir                  = f"outputs_w{WARMUP_FIXED}",
        report_to                   = "none",
        warmup_steps                = WARMUP_FIXED,
        max_steps                   = MAX_STEPS,
        learning_rate               = 2e-4,
        fp16                        = not is_bfloat16_supported(),
        bf16                        = is_bfloat16_supported(),
        logging_steps               = 1,
        optim                       = "adamw_8bit",
    ),
)

print(f"\n========== Training (warmup_steps = {WARMUP_FIXED}) ==========")
trainer.train()

results = []
for temp in TEMPERATURES:
    acc, jac = evaluate_metrics(lora_model, tokenizer, val_ds, temp=temp)
    label = "greedy" if temp is None else temp
    results.append({
        "temperature" : label,
        "exact_match" : round(acc*100, 2),
        "jaccard"     : round(jac*100, 2),
    })
    print(f">> temp={label} | Acc={acc:.2%} | Jac={jac:.2%}")

df = pd.DataFrame(results)
display(df)



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
==((====))==  Unsloth 2025.7.9: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/7000 [00:00<?, ? examples/s]


========== Training (warmup_steps = 200) ==========


...

100%|██████████| 100/100 [02:17<00:00,  1.37s/it]


>> temp=0.6 | Acc=4.00% | Jac=49.38%


100%|██████████| 100/100 [02:19<00:00,  1.40s/it]

>> temp=1.0 | Acc=2.00% | Jac=47.25%


temperature  exact_match  jaccard
0      greedy          9.0    55.49
1         0.2          8.0    54.19
2         0.4          7.0    52.59
3         0.6          4.0    49.38
4         1.0          2.0    47.25

fine tune

In [ ]:
!pip uninstall -y -q unsloth
!pip install --upgrade --quiet --no-cache-dir \
    "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --quiet trl datasets accelerate bitsandbytes pandas tqdm

from unsloth import FastLanguageModel, is_bfloat16_supported
import torch, os, json, urllib.request
from tqdm import tqdm
import pandas as pd

MODEL_NAME  = "unsloth/Llama-3.2-1B-Instruct"
MAX_SEQ_LEN = 2048

base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name     = MODEL_NAME,
    max_seq_length = MAX_SEQ_LEN,
    load_in_4bit   = True,
)
base_model.eval()

TABLES_URL  = "https://raw.githubusercontent.com/taoyds/spider/master/tables.json"
TABLES_FILE = "./tables.json"
if not os.path.exists(TABLES_FILE):
    urllib.request.urlretrieve(TABLES_URL, TABLES_FILE)

with open(TABLES_FILE, "r") as f:
    tables_list = json.load(f)
tables = {t["db_id"]: t for t in tables_list}

def build_schema(db_id):
    t = tables[db_id]
    col2tbl = {
        i: (t["column_names_original"][i][0], t["column_names_original"][i][1])
        for i in range(len(t["column_types"]))
    }
    parts = []
    for tidx, tname in enumerate(t["table_names_original"]):
        cols = [
            f'"{c}" {t["column_types"][cid]}'
            for cid,(tbl,c) in col2tbl.items()
            if tbl==tidx and c!="*"
        ]
        pk = [
            f'"{t["column_names_original"][cid][1]}"'
            for cid in t["primary_keys"]
            if col2tbl[cid][0]==tidx
        ]
        parts.append(
            f'"{tname}" {", ".join(cols)}' +
            (f', primary key: {" ".join(pk)}' if pk else "")
        )
    return " [SEP] ".join(parts)

from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

tokenizer = get_chat_template(base_tokenizer, chat_template="llama-3.1")

def format_entries(entry):
    convs = [
        [
            {"role":"user","content": q},
            {"role":"assistant","content": sql}
        ]
        for q, sql in zip(entry["question"], entry["query"])
    ]
    texts = [
        tokenizer.apply_chat_template(c, tokenize=False, add_generation_prompt=False)
        for c in convs
    ]
    return {
        "text": texts,
        "db_id": entry["db_id"],
        "question": entry["question"],
        "query": entry["query"]
    }

train_ds = load_dataset("xlangai/spider", split="train")
val_ds   = load_dataset("xlangai/spider", split="validation")
train_ds = train_ds.map(format_entries, batched=True)
val_ds   = val_ds.map(format_entries,   batched=True)

EVAL_SUBSET = 100

def jaccard_similarity(a: str, b: str) -> float:
    a_set, b_set = set(a.lower().split()), set(b.lower().split())
    if not a_set and not b_set:
        return 1.0
    return len(a_set & b_set) / len(a_set | b_set)

@torch.no_grad()
def evaluate(model, tokenizer, dataset, num_samples=EVAL_SUBSET):
    model.eval()
    subset = dataset.shuffle(seed=42).select(range(num_samples))
    exact = 0
    jac_sum = 0.0

    for item in tqdm(subset, total=num_samples):
        schema_txt = build_schema(item["db_id"])
        system_msg = f"Database schema:\n{schema_txt}"
        user_msg   = item["question"]

        # قالب چت با system + user
        prompt_ids = tokenizer.apply_chat_template(
            [
              {"role":"system",     "content": system_msg},
              {"role":"user",       "content": user_msg}
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to("cuda")

        out = model.generate(
            input_ids      = prompt_ids,
            attention_mask = torch.ones_like(prompt_ids),
            max_new_tokens = 64,
            do_sample      = False,
        )

        gen = out[0, prompt_ids.shape[1]:]
        pred = tokenizer.decode(gen, skip_special_tokens=True)
        gt   = item["query"]

        if pred.strip() == gt.strip():
            exact += 1
        jac_sum += jaccard_similarity(pred, gt)

    return exact/num_samples, jac_sum/num_samples

base_acc, base_jac = evaluate(base_model, tokenizer, val_ds)
print(f"\n>>> Base model | Acc={base_acc:.2%} | Jac={base_jac:.2%}")

from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer

lora_model = FastLanguageModel.get_peft_model(
    base_model,
    r                         = 4,
    target_modules            = ["q_proj","k_proj","v_proj","o_proj",
                                 "gate_proj","up_proj","down_proj"],
    lora_alpha                = 16,
    lora_dropout              = 0,
    bias                      = "none",
    use_gradient_checkpointing= "unsloth",
    random_state              = 3407,
    use_rslora                = False,
)

trainer = SFTTrainer(
    model              = lora_model,
    tokenizer          = tokenizer,
    train_dataset      = train_ds,
    dataset_text_field = "text",
    max_seq_length     = MAX_SEQ_LEN,
    data_collator      = DataCollatorForSeq2Seq(tokenizer=tokenizer),
    packing            = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        output_dir        = "outputs_r4",
        learning_rate     = 2e-4,
        max_steps         = 120,
        warmup_steps      = 200,
        weight_decay      = 0.01,
        lr_scheduler_type = "linear",
        report_to         = "none",
        fp16              = not is_bfloat16_supported(),
        bf16              = is_bfloat16_supported(),
        logging_steps     = 1,
        optim             = "adamw_8bit",
        seed              = 3407,
    ),
)

print("\n========== Fine-tuning (LoRA) ==========")
trainer.train()

ft_acc, ft_jac = evaluate(lora_model, tokenizer, val_ds)
print(f"\n>>> Fine-tuned | Acc={ft_acc:.2%} | Jac={ft_jac:.2%}")

df = pd.DataFrame([
    ["Base",            round(base_acc*100,2), round(base_jac*100,2)],
    ["Fine-tuned",  round(ft_acc*100,2),   round(ft_jac*100,2)]
], columns=["Model","Exact-match %","Jaccard %"])
display(df)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
==((====))==  Unsloth 2025.7.9: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

100%|██████████| 100/100 [02:41<00:00,  1.61s/it]



>>> Base model | Acc=0.00% | Jac=9.09%


...


========== Fine-tuning (LoRA) ==========


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 2,818,048 of 1,238,632,448 (0.23% trained)


100%|██████████| 100/100 [02:12<00:00,  1.32s/it]


>>> Fine-tuned | Acc=11.00% | Jac=56.20%


Model  Exact-match %  Jaccard %
0        Base            0.0       9.09
1  Fine-tuned           11.0      56.20

1

In [ ]:
!pip uninstall -y -q unsloth
!pip install --quiet --upgrade --no-cache-dir \
    "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --quiet datasets accelerate bitsandbytes tqdm

from unsloth import FastLanguageModel
import torch
from tqdm import tqdm

MODEL_NAME  = "unsloth/Llama-3.2-1B-Instruct"
MAX_SEQ_LEN = 2048

base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name     = MODEL_NAME,
    max_seq_length = MAX_SEQ_LEN,
    load_in_4bit   = True,
)
base_model.eval()  
import os, json, urllib.request, random

TABLES_URL  = "https://raw.githubusercontent.com/taoyds/spider/master/tables.json"
TABLES_FILE = "./tables.json"
if not os.path.exists(TABLES_FILE):
    urllib.request.urlretrieve(TABLES_URL, TABLES_FILE)

with open(TABLES_FILE, "r") as f:
    tables_list = json.load(f)
tables = {t["db_id"]: t for t in tables_list}

def build_schema(db_id: str) -> str:
    t = tables[db_id]
    col2tbl = {
        i: (t["column_names_original"][i][0], t["column_names_original"][i][1])
        for i in range(len(t["column_types"]))
    }
    parts = []
    for tidx, tname in enumerate(t["table_names_original"]):
        cols = [
            f'"{c}" {t["column_types"][cid]}'
            for cid,(tbl,c) in col2tbl.items()
            if tbl==tidx and c!="*"
        ]
        pk = [
            f'"{t["column_names_original"][cid][1]}"'
            for cid in t["primary_keys"]
            if col2tbl[cid][0]==tidx
        ]
        part = f'"{tname}" ' + ", ".join(cols)
        if pk:
            part += f", primary key: {' '.join(pk)}"
        parts.append(part)
    return " [SEP] ".join(parts)

from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(base_tokenizer, chat_template="llama-3.1")

def nl_to_sql(question: str, *, schema_txt: str = None, max_new_tokens: int = 128) -> str:
    messages = []
    if schema_txt:
        messages.append({
            "role": "system",
            "content": f"Database schema:\n{schema_txt}"
        })
    messages.append({"role": "user", "content": question})

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    with torch.no_grad():
        out = base_model.generate(
            input_ids      = prompt,
            attention_mask = torch.ones_like(prompt),
            max_new_tokens = max_new_tokens,
            do_sample      = False,
        )
    gen = out[0, prompt.shape[1]:]
    return tokenizer.decode(gen, skip_special_tokens=True).strip()

from datasets import load_dataset

val_ds = load_dataset("xlangai/spider", split="validation")

def jaccard(a: str, b: str) -> float:
    A, B = set(a.lower().split()), set(b.lower().split())
    if not A and not B:
        return 1.0
    return len(A & B) / len(A | B)

@torch.no_grad()
def evaluate_on_spider(model, tokenizer, dataset, n_samples: int = 100):
    correct = 0
    jac_sum = 0.0
    subset = dataset.shuffle(seed=42).select(range(n_samples))

    for item in tqdm(subset, total=n_samples):
        raw_q      = item["question"]
        schema_txt = build_schema(item["db_id"])
        pred_sql   = nl_to_sql(raw_q, schema_txt=schema_txt, max_new_tokens=128)
        gt_sql     = item["query"]

        if pred_sql.strip() == gt_sql.strip():
            correct += 1
        jac_sum += jaccard(pred_sql, gt_sql)

    acc = correct / n_samples
    jac = jac_sum  / n_samples
    return acc, jac

base_acc, base_jac = evaluate_on_spider(base_model, tokenizer, val_ds, n_samples=100)
print(f"\nBase model on Spider validation (100 samples):")
print(f"  Exact-match accuracy: {base_acc:.2%}")
print(f"  Jaccard similarity : {base_jac:.2%}")

custom_question = "List all students who have registered for more than three courses."
custom_db_id    = "student_transcripts_tracking"  # یکی از پایگاه‌های Spider مناسب برای این سؤال
custom_schema   = build_schema(custom_db_id)
custom_pred_sql = nl_to_sql(custom_question, schema_txt=custom_schema)

print("\n— Custom example —")
print("Question :", custom_question)
print("Predicted SQL:\n", custom_pred_sql)

print("\n— Three random Spider samples —")
sample_items = val_ds.shuffle(seed=2025).select(range(3))

for idx, item in enumerate(sample_items):
    q   = item["question"]
    db  = item["db_id"]
    gt  = item["query"]
    sc  = build_schema(db)
    prd = nl_to_sql(q, schema_txt=sc)

    print(f"\nSample {idx+1}")
    print("DB ID        :", db)
    print("Question     :", q)
    print("Model output :", prd)
    print("Ground truth :", gt)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
==((====))==  Unsloth 2025.7.9: Fast Llama patching. Transformers: 4.53.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/831k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

100%|██████████| 100/100 [04:46<00:00,  2.86s/it]



Base model on Spider validation (100 samples):
  Exact-match accuracy: 0.00%
  Jaccard similarity : 10.01%

— Custom example —
Question : List all students who have registered for more than three courses.
Predicted SQL:
 To list all students who have registered for more than three courses, you can use the following SQL query:

```sql
SELECT *
FROM Students
WHERE student_id IN (
  SELECT student_id
  FROM Student_Enrolment
  GROUP BY student_id
  HAVING COUNT(course_id) > 3
)
```

This query works as follows:

- It selects all columns (`*`) from the `Students` table.
- It uses a subquery to find the `student_id` for each course.
- It groups the results by `student_id` using the `GROUP BY` clause.
- It uses the `

— Three random Spider samples —

Sample 1
DB ID        : tvshow
Question     : What are all of the episodes ordered by ratings?
Model output : To retrieve the episodes ordered by ratings, you can use the following SQL query:

```sql
SELECT 
    T1.Episode, 
    T1.Rating, 
   

3-prompt engineering

In [ ]:
!pip uninstall -y -q unsloth
!pip install --quiet --upgrade --no-cache-dir \
    "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --quiet trl datasets accelerate bitsandbytes pandas tqdm

from unsloth import FastLanguageModel
import torch, os, json, urllib.request, random
import pandas as pd
from tqdm import tqdm

MODEL_NAME  = "unsloth/Llama-3.2-1B-Instruct"
MAX_SEQ_LEN = 2048

base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name     = MODEL_NAME,
    max_seq_length = MAX_SEQ_LEN,
    load_in_4bit   = True,
)
base_model.eval()

from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

tokenizer = get_chat_template(base_tokenizer, chat_template="llama-3.1")

def to_chat(entry):
    convs = [
        [{"role":"user","content":q},{"role":"assistant","content":sql}]
        for q,sql in zip(entry["question"], entry["query"])
    ]
    texts = [
        tokenizer.apply_chat_template(c, tokenize=False, add_generation_prompt=False)
        for c in convs
    ]
    return {
        "text": texts,
        "db_id": entry["db_id"],
        "question": entry["question"],
        "query": entry["query"]
    }

train_ds = load_dataset("xlangai/spider", split="train").map(to_chat, batched=True)
val_ds   = load_dataset("xlangai/spider", split="validation").map(to_chat, batched=True)

TABLES_URL  = "https://raw.githubusercontent.com/taoyds/spider/master/tables.json"
TABLES_FILE = "./tables.json"
if not os.path.exists(TABLES_FILE):
    urllib.request.urlretrieve(TABLES_URL, TABLES_FILE)

with open(TABLES_FILE) as f:
    tables_list = json.load(f)
tables = {t["db_id"]: t for t in tables_list}

def build_schema(db_id):
    t = tables[db_id]
    col2tbl = {
        i:(t["column_names_original"][i][0], t["column_names_original"][i][1])
        for i in range(len(t["column_types"]))
    }
    parts = []
    for tidx, tname in enumerate(t["table_names_original"]):
        cols = [
            f'"{c}" {t["column_types"][cid]}'
            for cid,(tbl,c) in col2tbl.items() if tbl==tidx and c!="*"
        ]
        pk   = [
            f'"{t["column_names_original"][cid][1]}"'
            for cid in t["primary_keys"] if col2tbl[cid][0]==tidx
        ]
        part = f'"{tname}" ' + ", ".join(cols)
        if pk:
            part += f", primary key: {' '.join(pk)}"
        parts.append(part)
    return " [SEP] ".join(parts)

INSTRUCTION = (
    "You are an AI assistant that converts natural-language questions into SQL. "
    "Return ONLY the raw SQL query, with no explanation, comments, formatting, "
    "or markdown fences. Do not output anything except the SQL statement."
)

def pick_examples(db_id, k=1, seed=3407):
    random.seed(seed)
    same_db = train_ds.filter(lambda ex: ex["db_id"]==db_id)
    exs = []
    if len(same_db)>0:
        idxs = random.sample(range(len(same_db)), min(k, len(same_db)))
        exs = [(same_db[i]["question"], same_db[i]["query"]) for i in idxs]
    if len(exs)<k:                                    
        idxs = random.sample(range(len(train_ds)), k-len(exs))
        exs += [(train_ds[i]["question"], train_ds[i]["query"]) for i in idxs]
    return exs

def build_prompt(question, db_id, demos=None, schema_txt=None):
    conv = [{"role":"system","content":INSTRUCTION}]
    if schema_txt:
        conv.append({"role":"system","content":f"Database schema:\n{schema_txt}"})
    if demos:
        for q,a in demos:
            conv.extend([
                {"role":"user","content":q},
                {"role":"assistant","content":a}
            ])
    conv.append({"role":"user","content":question})
    return tokenizer.apply_chat_template(
        conv, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to("cuda")

def jaccard(a,b):
    A,B = set(a.lower().split()), set(b.lower().split())
    return 1.0 if not A and not B else len(A&B)/len(A|B)

@torch.no_grad()
def evaluate_variant(variant, n=100):
    exact = jac = 0
    subset = val_ds.shuffle(seed=42).select(range(n))

    for item in tqdm(subset, total=n):
        db, q      = item["db_id"], item["question"]
        schema_txt = build_schema(db)

        if variant=="schema_full":
            pt = build_prompt(q, db,
                              demos=pick_examples(db,3),
                              schema_txt=schema_txt)
            gen_kwargs = dict(max_new_tokens=96, do_sample=False)
        else:
            if variant=="zero":
                pt = build_prompt(q, db)
            elif variant=="one":
                pt = build_prompt(q, db, demos=pick_examples(db,1))
            elif variant=="few":
                pt = build_prompt(q, db, demos=pick_examples(db,3))
            elif variant=="schema_summary":
                summary = " | ".join(
                    f'{tname}: ' +
                    ", ".join(c for (tbl,c) in tables[db]["column_names_original"]
                              if tbl==idx and c!="*")
                    for idx,tname in enumerate(tables[db]["table_names_original"])
                )
                pt = build_prompt(q, db, schema_txt=summary)
            elif variant=="schema_semantic":
                sem = " ".join(
                    f'Table "{tname}" contains ' +
                    ", ".join(c.replace("_"," ") for (tbl,c) in tables[db]["column_names_original"]
                              if tbl==idx and c!="*") + "."
                    for idx,tname in enumerate(tables[db]["table_names_original"])
                )
                pt = build_prompt(q, db, schema_txt=sem)
            else:
                raise ValueError(variant)

            gen_kwargs = dict(
                max_new_tokens=64,
                do_sample=True,
                temperature=0.8,
                top_p=0.95
            )

        out = base_model.generate(
            input_ids      = pt,
            attention_mask = torch.ones_like(pt),
            **gen_kwargs
        )

        pred = tokenizer.decode(out[0, pt.shape[1]:], skip_special_tokens=True)
        gt   = item["query"]

        if pred.strip()==gt.strip():
            exact += 1
        jac += jaccard(pred, gt)

    return exact/n, jac/n

strategies = [
    "zero","one","few",
    "schema_full","schema_summary","schema_semantic"
]

results = []
for st in strategies:
    print(f"\n🧪 Evaluating {st}")
    acc, jac = evaluate_variant(st, n=100)
    results.append([st, round(acc*100,2), round(jac*100,2)])
    print(f" ↳ Acc={acc:.2%}, Jac={jac:.2%}")

pd.DataFrame(results, columns=["Prompt strategy","Exact-match %","Jaccard %"])


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
==((====))==  Unsloth 2025.7.9: Fast Llama patching. Transformers: 4.53.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]


🧪 Evaluating zero


100%|██████████| 100/100 [01:01<00:00,  1.62it/s]


...

100%|██████████| 100/100 [01:09<00:00,  1.43it/s]


 ↳ Acc=1.00%, Jac=40.67%

🧪 Evaluating schema_semantic


100%|██████████| 100/100 [01:21<00:00,  1.23it/s]

 ↳ Acc=1.00%, Jac=41.16%


Prompt strategy  Exact-match %  Jaccard %
0             zero            0.0      22.78
1              one            0.0      25.63
2              few            0.0      31.85
3      schema_full            2.0      44.01
4   schema_summary            1.0      40.67
5  schema_semantic            1.0      41.16

In [ ]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 33.7 MB/s eta 0:00:00


In [ ]:
!pip install -U datasets          # Hugging Face Datasets library (load_dataset)
!pip install -U pyarrow           # fast columnar data serialization
!pip install -U fsspec            # filesystem abstraction


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

temp on T5-LM-Large-text2sql-spider

In [ ]:
# pip install transformers bitsandbytes accelerate datasets pandas tqdm

import json
import re
import pathlib
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
import pandas as pd
from tqdm import tqdm

MODEL_ID     = "gaussalgo/T5-LM-Large-text2sql-spider"
TABLE_PATH   = pathlib.Path("./tables.json")
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"
EVAL_SUBSET  = 100  
MAX_NEW_TOKS = 64   
OUTPUT_CSV   = "eval_text2sql_spider_temps.csv"
# None → greedy
TEMPERATURES = [None, 0.2, 0.4, 0.6, 1.0]

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
print(f"Loading model {MODEL_ID} in 4-bit...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"
)
model.eval()

print("Loading Spider dataset...")
ds = load_dataset("spider")
val_ds = ds["validation"].shuffle(seed=42).select(range(EVAL_SUBSET))

if not TABLE_PATH.exists():
    raise FileNotFoundError(f"Schema file not found: {TABLE_PATH}")
tables = {t["db_id"]: t for t in json.load(open(TABLE_PATH, encoding='utf-8'))}

def build_schema(db_id: str) -> str:
    t = tables[db_id]
    col2tbl = {i: (c[0], c[1]) for i, c in enumerate(t["column_names_original"])}
    parts = []
    for tidx, tbl in enumerate(t["table_names_original"]):
        cols = []
        for cid, (tbl_idx, col) in col2tbl.items():
            if tbl_idx == tidx and col != '*':
                ctype = t["column_types"][cid]
                cols.append(f'"{col}" {ctype}')
        pk = [f'"{t["column_names_original"][cid][1]}"'
              for cid in t["primary_keys"] if col2tbl[cid][0] == tidx]
        pk_txt = f', primary key: {" ".join(pk)}' if pk else ''
        parts.append(f'"{tbl}" ' + ' , '.join(cols) + pk_txt)
    return ' [SEP] '.join(parts)


def normalize_sql(sql: str) -> str:
    text = re.sub(r"\s+", ' ', sql.lower())
    return text.replace(' ,', ',').replace(' .', '.').strip()


def jaccard_similarity(pred: str, gold: str) -> float:
    set_p = set(normalize_sql(pred).split())
    set_g = set(normalize_sql(gold).split())
    if not set_p and not set_g:
        return 1.0
    return len(set_p & set_g) / len(set_p | set_g)

print("Starting evaluation across temperatures...")
all_results = []

for temp in TEMPERATURES:
    label = "greedy" if temp is None else str(temp)
    print(f"\nEvaluating temperature: {label}")
    correct = 0
    jacc_sum = 0.0
    for item in tqdm(val_ds, desc=f"Temp={label}", ncols=80):
        q    = item["question"]
        gold = item["query"]
        db   = item["db_id"]

        schema_text = build_schema(db)
        prompt = f"Question: {q} Schema: {schema_text}"

        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKS,
                do_sample=(temp is not None),
                temperature=(temp if temp is not None else 1.0),
                num_beams=1
            )
        pred = tokenizer.decode(out[0], skip_special_tokens=True)

        is_correct = normalize_sql(pred) == normalize_sql(gold)
        correct += int(is_correct)
        jacc_sum += jaccard_similarity(pred, gold)

        all_results.append({
            "temperature": label,
            "question": q,
            "db_id": db,
            "gold_sql": gold,
            "pred_sql": pred,
            "exact": is_correct,
            "jaccard": jaccard_similarity(pred, gold)
        })

    acc = correct / len(val_ds)
    avg_jacc = jacc_sum / len(val_ds)
    print(f">> temp={label} | Exact Match: {acc:.2%} | Avg Jaccard: {avg_jacc:.2%}")

df_all = pd.DataFrame(all_results)
df_all.to_csv(OUTPUT_CSV.replace('.csv', '_details.csv'), index=False)

summary = df_all.groupby('temperature').agg(
    exact_match=('exact', 'mean'),
    avg_jaccard=('jaccard', 'mean')
).reset_index()
summary.to_csv(OUTPUT_CSV, index=False)
print(f"Detailed results saved to {OUTPUT_CSV.replace('.csv', '_details.csv')}")
print(f"Summary saved to {OUTPUT_CSV}")


Loading model gaussalgo/T5-LM-Large-text2sql-spider in 4-bit...
Loading Spider dataset...
Starting evaluation across temperatures...

Evaluating temperature: greedy


Temp=greedy: 100%|████████████████████████████| 100/100 [03:57<00:00,  2.37s/it]


>> temp=greedy | Exact Match: 24.00% | Avg Jaccard: 73.36%

Evaluating temperature: 0.2


Temp=0.2: 100%|███████████████████████████████| 100/100 [04:02<00:00,  2.42s/it]


>> temp=0.2 | Exact Match: 25.00% | Avg Jaccard: 73.29%

Evaluating temperature: 0.4


...

>> temp=0.4 | Exact Match: 24.00% | Avg Jaccard: 72.69%

Evaluating temperature: 0.6


Temp=0.6: 100%|███████████████████████████████| 100/100 [03:57<00:00,  2.38s/it]


>> temp=0.6 | Exact Match: 23.00% | Avg Jaccard: 71.06%

Evaluating temperature: 1.0


Temp=1.0: 100%|███████████████████████████████| 100/100 [03:56<00:00,  2.37s/it]

>> temp=1.0 | Exact Match: 24.00% | Avg Jaccard: 70.61%
Detailed results saved to eval_text2sql_spider_temps_details.csv
Summary saved to eval_text2sql_spider_temps.csv


In [ ]:
print(summary)

  temperature  exact_match  avg_jaccard
0         0.2         0.25     0.732854
1         0.4         0.24     0.726920
2         0.6         0.23     0.710621
3         1.0         0.24     0.706125
4      greedy         0.24     0.733599


4-RAG

In [ ]:
!pip uninstall -y -q unsloth
!pip install --quiet --upgrade --no-cache-dir \
  "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --quiet sentence-transformers faiss-cpu datasets accelerate \
  bitsandbytes pandas tqdm

from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import torch, json, os, urllib.request, random, faiss, numpy as np, pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

MODEL_NAME = "unsloth/Llama-3.2-1B-Instruct"
MAX_LEN    = 2048

base_model, base_tok = FastLanguageModel.from_pretrained(
    model_name     = MODEL_NAME,
    max_seq_length = MAX_LEN,
    load_in_4bit   = True,           # 4-bit برای صرفه‌جویی در حافظه GPU
)
base_model.eval()
tokenizer = get_chat_template(base_tok, chat_template="llama-3.1")

train_ds = load_dataset("xlangai/spider", split="train")
val_ds   = load_dataset("xlangai/spider", split="validation")

TABLES_URL  = "https://raw.githubusercontent.com/taoyds/spider/master/tables.json"
TABLES_FILE = "./tables.json"
if not os.path.exists(TABLES_FILE):
    urllib.request.urlretrieve(TABLES_URL, TABLES_FILE)

with open(TABLES_FILE) as f:
    tables = {t["db_id"]: t for t in json.load(f)}

def build_docs(db_id: str):
    """هر جدول → یک سند متنی خوانا (برای ایندکس FAISS)"""
    t = tables[db_id]
    docs = []
    for idx, tname in enumerate(t["table_names_original"]):
        cols  = [c for (tbl,c) in t["column_names_original"] if tbl == idx and c != "*"]
        types = [
            t["column_types"][i]
            for i,(tbl,_) in enumerate(t["column_names_original"])
            if tbl == idx and _ != "*"
        ]
        col_str = ", ".join(f"{c} ({tp})" for c, tp in zip(cols, types))
        docs.append(f"Table {tname}: {col_str}")
    return docs

EMB_MODELS = {
    "MiniLM": "sentence-transformers/all-MiniLM-L6-v2",
    "BGE"   : "BAAI/bge-base-en-v1.5"
}

faiss_indices, doc_store, embedders = {}, {}, {}

for tag, emb_name in EMB_MODELS.items():
    print(f" Building FAISS index for {tag} (CPU)…")
    embedder = SentenceTransformer(emb_name, device="cpu")       
    embedders[tag] = embedder
    idx_dict, store = {}, {}

    for db_id in tqdm(tables.keys(), leave=False):
        docs = build_docs(db_id)
        emb  = embedder.encode(docs, convert_to_numpy=True, show_progress_bar=False)
        faiss.normalize_L2(emb)                                
        index = faiss.IndexFlatIP(emb.shape[1])
        index.add(emb)
        idx_dict[db_id] = index
        store[db_id]    = docs

    faiss_indices[tag] = idx_dict
    doc_store[tag]     = store

# پاک‌سازی حافظه GPU (فقط احتیاط)
import gc, torch
gc.collect(); torch.cuda.empty_cache()

SYSTEM_MSG = (
    "You are an AI assistant that converts natural-language questions into SQL. "
    "Return ONLY the raw SQL query, with no explanation, comments, or markdown."
)

def build_prompt(question: str, retrieved_docs: str = None):
    conv = [{"role": "system", "content": SYSTEM_MSG}]
    if retrieved_docs:
        conv.append({"role": "system", "content": "Context:\n" + retrieved_docs})
    conv.append({"role": "user", "content": question})

    return tokenizer.apply_chat_template(
        conv,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

def jaccard(a: str, b: str):
    A, B = set(a.lower().split()), set(b.lower().split())
    return 1.0 if not A and not B else len(A & B) / len(A | B)

@torch.no_grad()
def evaluate(mode="zero", emb_tag=None, k=3, n=100):
   
    exact = jac = 0.0
    subset = val_ds.shuffle(seed=42).select(range(n))

    for ex in tqdm(subset, total=n, leave=False):
        db_id, q, gt = ex["db_id"], ex["question"], ex["query"]

        if mode == "zero":
            prompt = build_prompt(q)

        elif mode == "schema_full":
            schema_text = " [SEP] ".join(build_docs(db_id))
            prompt = build_prompt(q, schema_text)

        elif mode == "rag":
            embedder = embedders[emb_tag]
            q_emb    = embedder.encode([q], convert_to_numpy=True)
            faiss.normalize_L2(q_emb)
            idx      = faiss_indices[emb_tag][db_id]
            _, I     = idx.search(q_emb, k)
            retrieved = " [SEP] ".join(doc_store[emb_tag][db_id][i] for i in I[0])
            prompt    = build_prompt(q, retrieved)

        else:
            raise ValueError(mode)

        out  = base_model.generate(
            input_ids      = prompt,
            attention_mask = torch.ones_like(prompt),
            max_new_tokens = 96,
            do_sample      = False
        )
        pred = tokenizer.decode(out[0, prompt.shape[1]:], skip_special_tokens=True)

        if pred.strip() == gt.strip():
            exact += 1
        jac += jaccard(pred, gt)

    return exact / n, jac / n

results = []

print("🧪 Evaluating ZERO-shot …")
acc, jac = evaluate(mode="zero", n=100)
results.append(["Zero-shot (no schema)", acc * 100, jac * 100])

print("🧪 Evaluating FULL-SCHEMA …")
acc, jac = evaluate(mode="schema_full", n=100)
results.append(["Schema-Full", acc * 100, jac * 100])

print("🧪 Evaluating RAG + MiniLM …")
acc, jac = evaluate(mode="rag", emb_tag="MiniLM", k=3, n=100)
results.append(["RAG + MiniLM", acc * 100, jac * 100])

print("🧪 Evaluating RAG + BGE …")
acc, jac = evaluate(mode="rag", emb_tag="BGE", k=3, n=100)
results.append(["RAG + BGE", acc * 100, jac * 100])

df = pd.DataFrame(results, columns=["Strategy", "Exact-match %", "Jaccard %"])
pd.set_option("display.precision", 2)
print("\n=== Results ===")
print(df.to_string(index=False))



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
==((====))==  Unsloth 2025.7.11: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
🔧 Building FAISS index for MiniLM (CPU)…


🔧 Building FAISS index for BGE (CPU)…


🧪 Evaluating ZERO-shot …


🧪 Evaluating FULL-SCHEMA …


🧪 Evaluating RAG + MiniLM …


🧪 Evaluating RAG + BGE …



=== Results ===
             Strategy  Exact-match %  Jaccard %
Zero-shot (no schema)            0.0      27.16
          Schema-Full            2.0      47.01
         RAG + MiniLM            2.0      44.87
            RAG + BGE            2.0      46.21


6- rag + fine-tune

In [ ]:
!pip uninstall -y -q unsloth faiss faiss-cpu
!pip install --quiet --upgrade --no-cache-dir \
    "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --quiet datasets accelerate bitsandbytes tqdm sentence-transformers faiss-cpu

from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

import torch, os, json, urllib.request, faiss, numpy as np
from tqdm import tqdm
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

MODEL_NAME   = "unsloth/Llama-3.2-1B-Instruct"
EMB_MODEL    = "BAAI/bge-large-en"         
MAX_SEQ_LEN  = 2048
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"
SEED         = 42

INSTRUCTION = (
    "You are an AI assistant that converts natural-language questions into SQL. "
    "Return ONLY the raw SQL query, with no explanation, comments, formatting, "
    "or markdown fences. Do not output anything except the SQL statement."
)

base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name     = MODEL_NAME,
    max_seq_length = MAX_SEQ_LEN,
    load_in_4bit   = True,
)
base_model.eval()
chat_tokenizer = get_chat_template(base_tokenizer, chat_template="llama-3.1")

TABLES_URL  = "https://raw.githubusercontent.com/taoyds/spider/master/tables.json"
TABLES_FILE = "./tables.json"
if not os.path.exists(TABLES_FILE):
    urllib.request.urlretrieve(TABLES_URL, TABLES_FILE)

SCHEMA_DICT = {t["db_id"]: t for t in json.load(open(TABLES_FILE))}

def build_schema(db_id: str) -> str:
    t = SCHEMA_DICT[db_id]
    col2tbl = {i: (t["column_names_original"][i][0],
                   t["column_names_original"][i][1])
               for i in range(len(t["column_types"]))}
    parts = []
    for tidx, tname in enumerate(t["table_names_original"]):
        cols = [f'"{c}" {t["column_types"][cid]}'
                for cid, (tbl, c) in col2tbl.items()
                if tbl == tidx and c != "*"]
        pk = [f'"{t["column_names_original"][cid][1]}"'
              for cid in t["primary_keys"] if col2tbl[cid][0] == tidx]
        seg = f'"{tname}" ' + ", ".join(cols)
        if pk: seg += f", primary key: {' '.join(pk)}"
        parts.append(seg)
    return " [SEP] ".join(parts)

print("⌛ Building FAISS index …")
train_ds = load_dataset("xlangai/spider", split="train")

emb_model = SentenceTransformer(EMB_MODEL, device=DEVICE)
all_q   = train_ds["question"]
all_sql = train_ds["query"]
all_db  = train_ds["db_id"]

bsz, vecs = 256, []
for i in tqdm(range(0, len(all_q), bsz)):
    vecs.append(emb_model.encode(all_q[i:i+bsz],
                                 convert_to_numpy=True, show_progress_bar=False))
embeddings = np.vstack(vecs).astype("float32")
faiss.normalize_L2(embeddings)
index = faiss.IndexFlatIP(embeddings.shape[1]); index.add(embeddings)
print("Vectors indexed:", index.ntotal)

def retrieve_examples(question: str, db_id: str, k: int = 5):
    qv = emb_model.encode([question], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(qv)
    _, I = index.search(qv, 20)        # زیاد می‌گیریم، بعد فیلتر
    sel = [idx for idx in I[0] if all_db[idx] == db_id][:k]
    return [{"question": all_q[i], "sql": all_sql[i]} for i in sel]

@torch.no_grad()
def nl_to_sql_rag(question, db_id, model, tokenizer,
                  schema_txt, k=5, max_new_tokens=128):
    msgs = [
        {"role":"system", "content": INSTRUCTION},
        {"role":"system", "content": f"Database schema:\n{schema_txt}"}
    ]
    for ex in retrieve_examples(question, db_id, k):
        msgs.append({"role":"system",
                     "content":f"Example Question: {ex['question']}\nExample SQL: {ex['sql']}"})
    msgs.append({"role":"user", "content": question})

    ids = tokenizer.apply_chat_template(
        msgs, tokenize=True, add_generation_prompt=True,
        return_tensors="pt").to(DEVICE)
    out = model.generate(ids, max_new_tokens=max_new_tokens, do_sample=False)
    return tokenizer.decode(out[0, ids.shape[1]:], skip_special_tokens=True).strip()

def jaccard(a, b):
    A, B = set(a.lower().split()), set(b.lower().split())
    return 1 if not A and not B else len(A&B)/len(A|B)

@torch.no_grad()
def evaluate(model, tok, ds, rag=True, n=100):
    sub = ds.shuffle(seed=SEED).select(range(n))
    acc=j_sum=0
    for it in tqdm(sub, total=n):
        sc = build_schema(it["db_id"])
        if rag:
            pr = nl_to_sql_rag(it["question"], it["db_id"], model, tok, sc)
        else:
            msgs = [
                {"role":"system","content":INSTRUCTION},
                {"role":"system","content":f"Database schema:\n{sc}"},
                {"role":"user","content":it["question"]},
            ]
            ids = tok.apply_chat_template(msgs, tokenize=True,
                                          add_generation_prompt=True,
                                          return_tensors="pt").to(DEVICE)
            pr = tok.decode(model.generate(ids, max_new_tokens=128,
                                           do_sample=False)[0,
                                ids.shape[1]:], skip_special_tokens=True).strip()
        acc += pr.strip()==it["query"].strip()
        j_sum += jaccard(pr, it["query"])
    return acc/n, j_sum/n

val_ds = load_dataset("xlangai/spider", split="validation")
print("\n⚡ Base + RAG …"); b_acc,b_j = evaluate(base_model,chat_tokenizer,val_ds,rag=True)
print(f"Base+RAG EM {b_acc:.2%} | Jaccard {b_j:.2%}")

from trl import SFTTrainer
from transformers import TrainingArguments

lora_cfg = dict(r=16,lora_alpha=32,lora_dropout=0.05,
                bias="none",
                target_modules=["q_proj","k_proj","v_proj",
                                "o_proj","gate_proj","up_proj","down_proj"])
ft_model = FastLanguageModel.get_peft_model(base_model, **lora_cfg)
ft_model.print_trainable_parameters()

TRAIN_SAMPLES = 7000
train_ds = train_ds = train_ds.shuffle(seed=SEED).select(range(TRAIN_SAMPLES))

def ffmt(ex):
    out=[]
    for q,sq,db in zip(ex["question"],ex["query"],ex["db_id"]):
        msgs=[{"role":"system","content":INSTRUCTION},
              {"role":"system","content":f"Database schema:\n{build_schema(db)}"},
              {"role":"user","content":q},
              {"role":"assistant","content":sq}]
        out.append(chat_tokenizer.apply_chat_template(msgs, tokenize=False))
    return {"text":out}

train_ds = train_ds.map(ffmt, batched=True,
                        remove_columns=train_ds.column_names)

trainer = SFTTrainer(
    model=ft_model, tokenizer=chat_tokenizer,
    train_dataset=train_ds, dataset_text_field="text",
    max_seq_length=MAX_SEQ_LEN,
    args=TrainingArguments(
        output_dir="./llama3_spider_lora",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        max_steps=1000,            
        learning_rate=2e-4,
        fp16=torch.cuda.is_available(),
        logging_steps=100,
        save_strategy="no",
        report_to="none",
    )
)

print("\n🚀 Fine-tuning …"); trainer.train()

print("\n⚡ Fine-tuned + RAG …")
ft_acc, ft_j = evaluate(ft_model, chat_tokenizer, val_ds, rag=True)
print(f"Fine-tuned+RAG EM {ft_acc:.2%} | Jaccard {ft_j:.2%}")

samples=[("List all students who have registered for more than three courses.",
          "student_transcripts_tracking"),
         ("What are the names of the authors who have published in more than two different journals?",
          "academic")]

for q,db in samples:
    print("\nQuestion:",q)
    print("SQL:", nl_to_sql_rag(q, db, ft_model, chat_tokenizer,
                                build_schema(db)))


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
==((====))==  Unsloth 2025.7.11: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
⌛ Building FAISS index …


100%|██████████| 28/28 [00:28<00:00,  1.03s/it]


Vectors indexed: 7000

⚡ Base + RAG …


100%|██████████| 100/100 [01:39<00:00,  1.00it/s]


Base+RAG EM 0.00% | Jaccard 38.85%
trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039


...


🚀 Fine-tuning …


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)



⚡ Fine-tuned + RAG …


  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 100/100 [02:14<00:00,  1.35s/it]


Fine-tuned+RAG EM 11.00% | Jaccard 62.19%

Question: List all students who have registered for more than three courses.
SQL: SELECT DISTINCT T1.first_name,  T1.last_name FROM students AS T1 JOIN student_enrolment AS T2 ON T1.student_id  =  T2.student_id JOIN student_course_enrolment AS T3 ON T2.student_enrolment_id  =  T3.student_enrolment_id GROUP BY T1.student_id HAVING count(*)  >  3

Question: What are the names of the authors who have published in more than two different journals?
SQL: SELECT T1.name FROM author AS T1 JOIN publication AS T2 ON T1.aid  =  T2.aid GROUP BY T1.name HAVING count(*)  >  2
